# Merge  Exposures list from Butler Spectractor Results to Find missing Spectractor results with (saved in csv file)

- author : Sylvie Dagoret-Campagne
- creation date : 2025-09-23 on run_v5
- last update : 2025-09-24 on run run_v_4

- read the list of exposures generated by TOOL_ListofExposures.ipynb notebook
- read the Spectractor Results produced by EXTR_viewSpectractorResults.ipynb notebook
- merge both by exposure id and save the merge in a npy file
- all path are given in `BUTLER00_parameters.py`

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

plt.rcParams["figure.figsize"] = (16,8)
plt.rcParams["axes.labelsize"] = 'xx-large'
plt.rcParams['axes.titlesize'] = 'xx-large'
plt.rcParams['xtick.labelsize']= 'xx-large'
plt.rcParams['ytick.labelsize']= 'xx-large'
plt.rcParams["legend.fontsize"] = "large"


In [ ]:
import ipywidgets as widgets
%matplotlib widget

In [ ]:
# CHECK THE CONFIG HERE !!!!!!
from BUTLER00_parameters import *

In [ ]:
DumpConfig()

## Configuration

In [ ]:
#FLAG_REPO_EMBARGO = True

In [ ]:
selected_run = "run_v4"

In [ ]:
atmfilename = extractedfilesdict[selected_run]
tag = legendtag[selected_run] 

In [ ]:
print(f"Spectractor Extracted atmospheric parameters file : {atmfilename}  for config {tag}")

In [ ]:
input_path = os.path.dirname(atmfilename)
inputfilename = os.path.basename(atmfilename)
inputfilename_root = inputfilename.split(".")[0] 
inputfilename_ext = inputfilename.split(".")[-1] 



In [ ]:
specdata = np.load(atmfilename,allow_pickle=True)

In [ ]:
df_spec = pd.DataFrame(specdata)

In [ ]:
df_spec

## List of Exposures

In [ ]:
if FLAG_REPO_EMBARGO:
    repo="/repo/embargo"
else:
    repo="/repo/main"
reponame = repo.replace("/","_")

path_exposureslist = "data/butlerregistry"
listexposures_file = os.path.join(path_exposureslist,"holosummary_all_filters" + reponame + ".csv")

In [ ]:
df_exp = pd.read_csv(listexposures_file,index_col=0)

if FLAG_REPO_EMBARGO:
    df_exp = df_exp[df_exp.day_obs> 20250000]
    

df_exp = df_exp.reset_index(drop=True)

In [ ]:
df_exp

In [ ]:
df_exp_prefixed = df_exp.rename(
    columns={col: f"ex_{col}" for col in df_exp.columns if col != "id"}
)

In [ ]:
df_exp_prefixed 

## Merge

In [ ]:
# Merge sur la colonne 'id'
df_spec_merged = pd.merge(
    df_exp_prefixed, 
    df_spec,
    on="id",        # clé de jointure, adapte selon ta colonne commune
    how="left"      # garde toutes les lignes de df_exp_prefixed
)

In [ ]:
if selected_run == "run_v5":
    df_spec_merged = df_spec_merged[ df_spec_merged["ex_day_obs"]> 20250100]
elif selected_run == "run_v6":
    df_spec_merged = df_spec_merged[ df_spec_merged["ex_day_obs"] <  20250100]

df_spec_merged = df_spec_merged.reset_index(drop=True)

In [ ]:
df_spec_merged["TARGET"]

## Save output

In [ ]:
outputfullfilename = mergedtofindmissings[selected_run] 
print(outputfullfilename)

In [ ]:
#os.makedirs(output_path, exist_ok=True)

In [ ]:
#print(outputfilename) 

In [ ]:
#outputfullfilename = os.path.join(output_path,outputfilename)

In [ ]:
rec_array = df_spec_merged.to_records(index=False)

In [ ]:
np.save(outputfullfilename , rec_array)

## Check

In [ ]:
newspecdata = np.load(outputfullfilename,allow_pickle=True)

In [ ]:
#newspecdata

In [ ]:
df_newspec = pd.DataFrame(newspecdata)

In [ ]:
df = df_newspec

In [ ]:
df = df.drop(df[df["ex_filter"] == "unknown~holo4_003"].index)
df = df.drop(df[df["ex_filter"] == "BG40_65mm_1~holo4_003"].index)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Comptage total par jour et filtre
counts_total = df.groupby(["ex_day_obs", "ex_filter"]).size().reset_index(name="total")

# Comptage des cas où TARGET est NaN
counts_nan = df[df["TARGET"].isna()].groupby(["ex_day_obs", "ex_filter"]).size().reset_index(name="nan_count")

# Fusionner
counts = pd.merge(counts_total, counts_nan, on=["ex_day_obs", "ex_filter"], how="left")
counts["nan_count"] = counts["nan_count"].fillna(0).astype(int)

# Préparation pour grouped bars
dates = sorted(counts["ex_day_obs"].unique())
filters = counts["ex_filter"].unique()
x = np.arange(len(dates))  # positions des dates
width = 0.3  # largeur des barres

fig, ax = plt.subplots(figsize=(20,6))

# On boucle sur chaque filtre
for i, filt in enumerate(filters):
    group = counts[counts["ex_filter"] == filt]

    # Aligner correctement avec x
    indices = [dates.index(d) for d in group["ex_day_obs"]]

    # Décalage pour séparer total et NaN
    ax.bar(
        x=np.array(indices) + i*width*1.5,  # position décalée par filtre
        height=group["total"],
        width=width,
        label=f"{filt} total",
        alpha=0.6
    )
    ax.bar(
        x=np.array(indices) + i*width*1.5 + width,  # décalage à droite pour les NaN
        height=group["nan_count"],
        width=width,
        label=f"{filt} NaN",
        alpha=0.9
    )

ax.set_xticks(x + width * len(filters) / 2)
ax.set_xticklabels(dates, rotation=90)
ax.set_ylabel("Nombre d'entrées")
ax.set_xlabel("Date (ex_day_obs)")
ax.set_title("Comptage par date et filtre (total vs TARGET NaN)")
ax.legend()
plt.tight_layout()
plt.show()


In [ ]:
dates = sorted(counts["ex_day_obs"].unique())
filters = counts["ex_filter"].unique()

# multiplier par 2 pour écarter les positions
x = np.arange(len(dates)) * 2  
width = 0.25  

fig, ax = plt.subplots(figsize=(20,6))  # plus large pour lisibilité

for i, filt in enumerate(filters):
    group = counts[counts["ex_filter"] == filt]
    indices = [dates.index(d) for d in group["ex_day_obs"]]

    # barres "total"
    ax.bar(
        x=np.array(indices)*2 + i*width*2,   # <-- décalage accentué
        height=group["total"],
        width=width,
        label=f"{filt} total",
        alpha=0.6
    )
    # barres "NaN"
    ax.bar(
        x=np.array(indices)*2 + i*width*2 + width,
        height=group["nan_count"],
        width=width,
        label=f"{filt} NaN",
        alpha=0.9
    )

ax.set_xticks(x + width * len(filters))  
ax.set_xticklabels(dates, rotation=90)
ax.set_ylabel("Nombre d'entrées")
ax.set_xlabel("Date (ex_day_obs)")
ax.set_title("Comptage par date et filtre (total vs TARGET NaN)")
ax.legend()
plt.tight_layout()
plt.show()


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Data : compter total et TARGET NaN
df_counts = (
    df.groupby(["ex_day_obs", "ex_filter"])
      .agg(total=("TARGET", "size"), nan_count=("TARGET", lambda x: x.isna().sum()))
      .reset_index()
)

plt.figure(figsize=(20,6))
sns.barplot(
    data=df_counts,
    x="ex_day_obs",
    y="total",
    hue="ex_filter",  # barres côte à côte par filtre
    dodge=True        # empêche l'empilement, sépare les barres
)

plt.xticks(rotation=90)
plt.ylabel("Nombre d'entrées")
plt.xlabel("Date (ex_day_obs)")
plt.title("Nombre d'entrées par date et filtre")
plt.tight_layout()
plt.show()


In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# 1️⃣ Compter total et NaN par jour et filtre
df_counts = (
    df.groupby(["ex_day_obs", "ex_filter"])
      .agg(
          total=("TARGET", "size"),
          nan_count=("TARGET", lambda x: x.isna().sum())
      )
      .reset_index()
)

# 2️⃣ Transformer en format “long” pour seaborn
df_long = df_counts.melt(
    id_vars=["ex_day_obs", "ex_filter"],
    value_vars=["total", "nan_count"],
    var_name="type",
    value_name="count"
)

# 3️⃣ Bar plot
plt.figure(figsize=(20,6))
sns.barplot(
    data=df_long,
    x="ex_day_obs",
    y="count",
    hue="type",
    dodge=True,
    palette=["skyblue", "salmon"]  # couleurs différentes pour total vs NaN
)

plt.xlabel("Date (ex_day_obs)")
plt.ylabel("Nombre d'entrées")
plt.title("Nombre d'entrées par date et filtre (total vs TARGET NaN)")
plt.xticks(rotation=90)
plt.legend(title="Type de comptage", loc="upper right")
plt.tight_layout()
plt.show()


In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# 1️⃣ Compter total et NaN par jour et filtre
df_counts = (
    df.groupby(["ex_day_obs", "ex_filter"])
      .agg(
          total=("TARGET", "size"),
          nan_count=("TARGET", lambda x: x.isna().sum())
      )
      .reset_index()
)

# 2️⃣ Transformer en format long
df_long = df_counts.melt(
    id_vars=["ex_day_obs", "ex_filter"],
    value_vars=["total", "nan_count"],
    var_name="type",
    value_name="count"
)

# 3️⃣ Bar plot avec filtre comme hue, type comme dodge
plt.figure(figsize=(14,6))
sns.barplot(
    data=df_long,
    x="ex_day_obs",
    y="count",
    hue="ex_filter",
    dodge=True,
    palette="tab10",
    ci=None
)

# Ajouter le type en hachures ou alpha pour distinguer total vs NaN
for i, bar in enumerate(plt.gca().patches):
    # alterner alpha selon type
    if i % 2 == 0:
        bar.set_alpha(0.6)   # total
    else:
        bar.set_alpha(1.0)   # NaN

plt.xlabel("Date (ex_day_obs)")
plt.ylabel("Nombre d'entrées")
plt.title("Nombre d'entrées par date et filtre (total vs TARGET NaN)")
plt.xticks(rotation=45)
plt.legend(title="FILTER")
plt.tight_layout()
plt.show()
